# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame


In [3]:
# import csv
csv_path = "output_data/cities.csv"
# read csv
weather_df = pd.read_csv(csv_path)
weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nome,64.5011,-165.4064,48.27,87,90,17.27,US,1628961563
1,1,arraial do cabo,-22.9661,-42.0278,66.16,88,90,5.75,BR,1628961563
2,2,dunedin,-45.8742,170.5036,49.80,47,64,11.01,NZ,1628961563
3,3,hobart,-42.8794,147.3294,55.26,63,90,5.01,AU,1628961282
4,4,ariquemes,-9.9133,-63.0408,99.34,21,22,3.00,BR,1628961564


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = weather_df["Humidity"].astype(float)

In [6]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
narrowed_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) \
                      & (weather_df["Wind Speed"] < 10) \
                      & (weather_df["Cloudiness"] == 0)].dropna()

narrowed_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
264,264,garavuti,37.5672,68.4469,75.85,16,0,5.01,TJ,1628961645
426,426,pozo colorado,-23.4923,-58.7972,79.47,22,0,4.07,PY,1628961698
474,474,xianyang,34.3378,108.7026,72.64,83,0,8.95,CN,1628961429
477,477,stanilesti,46.6183,28.1714,79.38,44,0,2.42,RO,1628961714


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = narrowed_weather_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
264,garavuti,TJ,37.5672,68.4469,
426,pozo colorado,PY,-23.4923,-58.7972,
474,xianyang,CN,34.3378,108.7026,
477,stanilesti,RO,46.6183,28.1714,


In [29]:
# find the closest hotel for each set of coordinates


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat, lon from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
#     print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params)
    
    # extract results
    results = response.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results["results"][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
264,garavuti,TJ,37.5672,68.4469,
426,pozo colorado,PY,-23.4923,-58.7972,
474,xianyang,CN,34.3378,108.7026,
477,stanilesti,RO,46.6183,28.1714,


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))

In [33]:
plt.savefig('hotel_heatmap.png')


<Figure size 432x288 with 0 Axes>